In [1]:
import numpy as np
import cv2 
import glob

In [2]:
import json

In [27]:

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*6,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('images/*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (6,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (6,6), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()



In [28]:
len(imgpoints)

9

In [12]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
np.save('images/calibration', [mtx, dist, rvecs, tvecs])

In [13]:
d = {'cam_col':  mtx}

In [14]:
mtx.tolist()

[[314.155773286397, 0.0, 147.5263698992761],
 [0.0, 316.2357234907263, 114.06164231122848],
 [0.0, 0.0, 1.0]]

In [23]:
data = {}
data['cam_col'] = mtx.tolist()


with open('cam_col.json', 'w') as outfile:
    json.dump(data, outfile)

In [15]:
img = cv2.imread('images/kek12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),0,(w,h))

In [25]:
roi

(0, 0, 320, 240)

In [16]:
import matplotlib.pylab as plt

In [17]:
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
#x,y,w,h = roi
#dst = dst[y:y+h, x:x+w]
cv2.imwrite('images/calibresult.png',dst)


True

In [28]:
# undistort
mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('images/calibresult.png',dst)

True

In [ ]:
import sys  

for arg in sys.argv:  
    print arg  